# Objetivo

Aplicar o K Nearest Neighbors Na base toda. 

# Método

A partir do conjunto reduzido de features (39) 100k rows, aplicar:
- fillna <b> OK </b>

- LabelEncoder (ou algo do tipo) <b> OK </b>
- Nearest Neighbors OK
    

# Leitura dos arquivos

In [2]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Leitura da lista de features a excluir

# Carregamento Clientes

## Redução no número de features



In [203]:
market = pd.read_csv('../../estaticos_market.csv')
port1 = pd.read_csv('../../clientes2_merge.csv')

In [204]:
exclude_df = pd.read_csv('exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)


143

In [205]:
port1.shape

(566, 182)

In [206]:
market.drop(columns=exclude_variables, inplace= True)
market.drop(columns=['Unnamed: 0'], inplace= True)
market.set_index('id',inplace=True)
market.shape

(462298, 37)

In [207]:
port1.drop(columns=exclude_variables, inplace= True)
port1.drop(columns=['Unnamed: 0'], inplace= True)
port1.set_index('id',inplace=True)
port1.shape

(566, 37)

In [208]:
#p1.to_csv('estaticos_49_features.csv')

# Managing NaNs


Para usar nos dados do portfólio:
- passar as strings para valor numérico
- passar bool para int
- fazer fillna para todos
- - em categoricas -> other
- - em numericas -> 0

Estamos usando o LabelEncoder

In [209]:
fill_dict = market.dtypes.to_dict()
cat2bool = ['fl_optante_simei','fl_optante_simples','fl_passivel_iss']

for feature in fill_dict:
    if(fill_dict[feature] == bool):
        fill_dict[feature] = False
    elif(fill_dict[feature] == object):
        fill_dict[feature] = 'other'
    else:
        fill_dict[feature] = 0
    if feature in cat2bool:
        fill_dict[feature] = False
        
fill_dict

{'fl_matriz': False,
 'de_natureza_juridica': 'other',
 'sg_uf': 'other',
 'natureza_juridica_macro': 'other',
 'de_ramo': 'other',
 'setor': 'other',
 'idade_empresa_anos': 0,
 'idade_emp_cat': 'other',
 'fl_me': False,
 'fl_sa': False,
 'fl_mei': False,
 'fl_ltda': False,
 'dt_situacao': 'other',
 'fl_st_especial': False,
 'fl_email': False,
 'fl_telefone': False,
 'fl_rm': 'other',
 'nm_divisao': 'other',
 'nm_segmento': 'other',
 'fl_optante_simples': False,
 'fl_optante_simei': False,
 'sg_uf_matriz': 'other',
 'de_saude_tributaria': 'other',
 'de_saude_rescencia': 'other',
 'nu_meses_rescencia': 0,
 'de_nivel_atividade': 'other',
 'empsetorcensitariofaixarendapopulacao': 0,
 'nm_meso_regiao': 'other',
 'nm_micro_regiao': 'other',
 'fl_passivel_iss': False,
 'qt_socios': 0,
 'idade_media_socios': 0,
 'qt_socios_feminino': 0,
 'de_faixa_faturamento_estimado': 'other',
 'de_faixa_faturamento_estimado_grupo': 'other',
 'vl_faturamento_estimado_aux': 0,
 'vl_faturamento_estimado_grupo

In [210]:
market.fillna(value=fill_dict, inplace=True)
market[cat2bool] = market[cat2bool].astype(bool)

bool2numeric = market.columns[market.dtypes == bool]
market[bool2numeric] = market[bool2numeric].astype(int)

In [211]:
#market_p1.head(2)

## Encoding

In [212]:
cat = market.columns[market.dtypes == object]
not_cat = market.columns[market.dtypes != object]

In [213]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [214]:
market_le = pd.DataFrame()
market_le[cat] = market[cat].apply(lambda col: le.fit_transform(col))
market_encoded = pd.concat([market[not_cat],market_le], axis =1)
market_encoded.dtypes

fl_matriz                                  int64
idade_empresa_anos                       float64
fl_me                                      int64
fl_sa                                      int64
fl_mei                                     int64
fl_ltda                                    int64
fl_st_especial                             int64
fl_email                                   int64
fl_telefone                                int64
fl_optante_simples                         int64
fl_optante_simei                           int64
nu_meses_rescencia                       float64
empsetorcensitariofaixarendapopulacao    float64
fl_passivel_iss                            int64
qt_socios                                float64
idade_media_socios                       float64
qt_socios_feminino                       float64
vl_faturamento_estimado_aux              float64
vl_faturamento_estimado_grupo_aux        float64
de_natureza_juridica                       int64
sg_uf               

Aqui fiz fiz um novo dataset para os ids do portifolio1 e o merge com os dados já aplicados o LabelEncoder

In [215]:
port1_le = pd.DataFrame(index=port1.index)
port1_le = market_encoded.merge(port1_le, how='right',left_index=True,right_index=True)


In [216]:
port1_le.head(3)

,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,fl_email,fl_telefone,fl_optante_simples,...,nm_divisao,nm_segmento,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
id,,,,,,,,,,,,,,,,,,,,,
09e95c1a84049001d086470a8f320a19b076f955a89122ff8360ac2e4d6eca5d,1,11.243836,0,0,0,0,0,1,1,0,...,0,0,0,4,0,0,17,57,6,6
dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728bb62194ca0842cce3e,1,43.789041,0,0,0,0,0,0,0,0,...,0,0,19,4,0,0,0,1,4,4
16843c9ffb920170477118933798510d8d5f00f5c56c1221773356355bded321,1,16.389041,0,0,0,0,0,0,1,0,...,36,11,16,4,0,0,9,11,3,3


# Nearest Neighbors em uma amostra do banco

Como não temos o que prever, usamos o treinamento não supervisionado do Nearest Neighbors



Apenas amostra por enquanto

Precisa escolher o numero de componentes a indicar

In [217]:
market_sample = market_encoded.sample(n=50000)

In [218]:
from sklearn.neighbors import NearestNeighbors

nearest = NearestNeighbors(n_neighbors=2, metric='cosine',radius = 5)

nearest.fit(market_sample)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=2, p=2, radius=5)

Dict para a lista de vizinhos mais próximos

In [219]:
neighbors_list = {}

for row in range(port1_le.shape[0]):
    #print(row)
    neighbors_list[row] = nearest.kneighbors(port1_le.iloc[[row]].values)

A resposta é uma lista da distância e uma lista de componentes. A componente é a linha do Dataset, ou seja, clientes.

In [220]:
neighbors_list

{0: (array([[2.86348723e-12, 3.53217455e-12]]), array([[41607, 16441]])),
 1: (array([[2.50133247e-13, 9.54458734e-13]]), array([[ 7227, 41607]])),
 2: (array([[2.20057306e-11, 6.28576080e-11]]), array([[22885, 40525]])),
 3: (array([[3.33066907e-16, 4.20885549e-13]]), array([[41607,  7227]])),
 4: (array([[2.16923146e-11, 6.31861230e-11]]), array([[22885, 40525]])),
 5: (array([[2.78096879e-10, 2.87627810e-10]]), array([[34247,  1626]])),
 6: (array([[6.65022482e-11, 7.55931984e-11]]), array([[36549, 33354]])),
 7: (array([[5.64600658e-07, 6.28193224e-07]]), array([[38014,  4116]])),
 8: (array([[4.97948238e-10, 1.76595291e-08]]), array([[38327, 21451]])),
 9: (array([[8.08988432e-11, 8.56533733e-11]]), array([[22885, 12492]])),
 10: (array([[4.04209999e-12, 1.03785869e-11]]), array([[30014, 27889]])),
 11: (array([[2.67412165e-06, 2.71859361e-06]]), array([[39673, 47469]])),
 12: (array([[1.15543575e-10, 3.48945983e-10]]), array([[16854, 17843]])),
 13: (array([[3.33066907e-16, 2.342

## Avaliação

Agora com o NN funcionando, precisa confrontar as sugestões para saber qual é o mais sugerido.

## Verificando o que foi mais sugerido

Da lista de vizinhos, listar em ordem os mais frequentes.

In [221]:
neighbors_list

{0: (array([[2.86348723e-12, 3.53217455e-12]]), array([[41607, 16441]])),
 1: (array([[2.50133247e-13, 9.54458734e-13]]), array([[ 7227, 41607]])),
 2: (array([[2.20057306e-11, 6.28576080e-11]]), array([[22885, 40525]])),
 3: (array([[3.33066907e-16, 4.20885549e-13]]), array([[41607,  7227]])),
 4: (array([[2.16923146e-11, 6.31861230e-11]]), array([[22885, 40525]])),
 5: (array([[2.78096879e-10, 2.87627810e-10]]), array([[34247,  1626]])),
 6: (array([[6.65022482e-11, 7.55931984e-11]]), array([[36549, 33354]])),
 7: (array([[5.64600658e-07, 6.28193224e-07]]), array([[38014,  4116]])),
 8: (array([[4.97948238e-10, 1.76595291e-08]]), array([[38327, 21451]])),
 9: (array([[8.08988432e-11, 8.56533733e-11]]), array([[22885, 12492]])),
 10: (array([[4.04209999e-12, 1.03785869e-11]]), array([[30014, 27889]])),
 11: (array([[2.67412165e-06, 2.71859361e-06]]), array([[39673, 47469]])),
 12: (array([[1.15543575e-10, 3.48945983e-10]]), array([[16854, 17843]])),
 13: (array([[3.33066907e-16, 2.342

In [243]:
list_size = len(neighbors_list)
num_neighbors = len(neighbors_list[0][1][0])

neighbors_idx_array = neighbors_list[0][1][0]
neighbors_distance_array = neighbors_list[0][0][0]
for line in range(1,list_size):
    neighbors_idx_array = np.concatenate((neighbors_idx_array, neighbors_list[line][1][0]), axis=None) 
    neighbors_distance_array = np.concatenate((neighbors_distance_array, neighbors_list[line][0][0]), axis=None) 

if len(neighbors_idx_array) != list_size*num_neighbors:
    print("ERROR: Check array size.")
# Temos agora um array unidimensional com os índices dos clientes recomendados

In [248]:
neighbors_distance_array[1]

3.5321745528449355e-12

Criar um dicionário com o número da linha, id e distância

In [289]:
dicio = {}
for idx,ind in zip(neighbors_idx_array, range(len(neighbors_idx_array))):
    dicio[ind] = (idx, market_sample.iloc[idx].name, neighbors_distance_array[ind])

In [290]:
dicio

{0: (41607,
  'ff045934d90aab184dd15e66daf3b3c97193bda5449298784b664681610609a0',
  2.8634872251132037e-12),
 1: (16441,
  'c24a6187673df4395d38fb1f16dce0f396db160d208b1d8cd5caf5c023fd1c69',
  3.5321745528449355e-12),
 2: (7227,
  '70f5bd78df544a353145a1bf8445abe15e9844e4ac0846c8ccc8ed40fd428963',
  2.5013324744804777e-13),
 3: (41607,
  'ff045934d90aab184dd15e66daf3b3c97193bda5449298784b664681610609a0',
  9.54458734270247e-13),
 4: (22885,
  'e8058f63f1bf4cc17deea56f96c84c690eb8291e1b41e6aa0824e0bb94d91fee',
  2.2005730571095228e-11),
 5: (40525,
  'ca7d4d28883a04fa6ffda83f022de343847cb45bc9879f4998d0b5a93ff01435',
  6.285760800750495e-11),
 6: (41607,
  'ff045934d90aab184dd15e66daf3b3c97193bda5449298784b664681610609a0',
  3.3306690738754696e-16),
 7: (7227,
  '70f5bd78df544a353145a1bf8445abe15e9844e4ac0846c8ccc8ed40fd428963',
  4.2088554863539684e-13),
 8: (22885,
  'e8058f63f1bf4cc17deea56f96c84c690eb8291e1b41e6aa0824e0bb94d91fee',
  2.1692314611243546e-11),
 9: (40525,
  'ca7d4d288

In [300]:
neig_df = pd.DataFrame.from_dict(dicio,orient='index')
neig_df.rename(columns={0:'row', 1:'id',2:'distance'},inplace = True)
neig_df.set_index('id', inplace=True)
#neig_df.row.value_counts()

In [301]:
conflicts = neig_df.merge(port1, how='inner',left_index=True,right_index=True)
suggestion_with_conflicts = neig_df.merge(market, how='left',left_index=True,right_index=True)
#neig_df.shape, port1.shape, conflicts.shape, suggestion_with_conflicts.shape, 
suggestions = suggestion_with_conflicts.drop(conflicts.index)

In [302]:
suggestions

,row,distance,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,...,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,idade_media_socios,qt_socios_feminino,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux
id,,,,,,,,,,,,,,,,,,,,,
00489ba3f0d05b82ff626d66bf32769dff5ba75303bc9b48f2e971e14369de79,29224,2.389955e-11,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,17.312329,15 a 20,...,OESTE MARANHENSE,IMPERATRIZ,1,3.0,58.000000,1.0,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",58058532.0,58058532.0
00489ba3f0d05b82ff626d66bf32769dff5ba75303bc9b48f2e971e14369de79,29224,8.879453e-12,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,17.312329,15 a 20,...,OESTE MARANHENSE,IMPERATRIZ,1,3.0,58.000000,1.0,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",58058532.0,58058532.0
00489ba3f0d05b82ff626d66bf32769dff5ba75303bc9b48f2e971e14369de79,29224,2.862877e-11,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,17.312329,15 a 20,...,OESTE MARANHENSE,IMPERATRIZ,1,3.0,58.000000,1.0,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",58058532.0,58058532.0
00489ba3f0d05b82ff626d66bf32769dff5ba75303bc9b48f2e971e14369de79,29224,2.988609e-12,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,17.312329,15 a 20,...,OESTE MARANHENSE,IMPERATRIZ,1,3.0,58.000000,1.0,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",58058532.0,58058532.0
00489ba3f0d05b82ff626d66bf32769dff5ba75303bc9b48f2e971e14369de79,29224,5.541567e-12,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,17.312329,15 a 20,...,OESTE MARANHENSE,IMPERATRIZ,1,3.0,58.000000,1.0,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",58058532.0,58058532.0
005924a1b48093ec3ca3aa7674c03b4f128ce40d694ed203cfa2e6ee2234ef0b,42747,5.578215e-07,1,SOCIEDADE EMPRESARIA LIMITADA,RO,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,30.709589,> 20,...,other,other,0,2.0,36.000000,2.0,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",3832790.5,4042790.5
005924a1b48093ec3ca3aa7674c03b4f128ce40d694ed203cfa2e6ee2234ef0b,42747,1.062474e-06,1,SOCIEDADE EMPRESARIA LIMITADA,RO,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,30.709589,> 20,...,other,other,0,2.0,36.000000,2.0,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",3832790.5,4042790.5
03691a50d397334a3cd9e7c004ed9ac669f7ce27d8f35f767e028881b0d5b80d,48828,1.267107e-07,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,COMERCIO POR ATACADO,COMERCIO,18.665753,15 a 20,...,NORTE MARANHENSE,AGLOMERACAO URBANA DE SAO LUIS,1,2.0,38.500000,1.0,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",3461875.2,4228248.0
0468615cfdca36ad1f90e672b0c03b138fafd784e1ba795b6c9fd8400505e2a1,15911,6.476716e-08,0,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,6.200000,5 a 10,...,OESTE POTIGUAR,MOSSORO,1,2.0,56.500000,0.0,"DE R$ 4.800.000,01 A R$ 10.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",5007355.0,13092936.0
